## <span style='font-family:Georgia'> Objectives
The purpose of this notebook is text mining, aimed at creating a baseline rule-model, which is to be a benchmark of the neural model developed in the later phase of the project.

In [1]:
# loading packages
import os
import re
import sys
import pandas as pd
import numpy as np
import morfeusz2
import seaborn as sns
import matplotlib.pyplot as plt

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from supportive_functions import (
    find_nonalpha,
    flatten,
    count_punctuation,
    separate_special_chars,
    rm_consecutive_spaces,
)

import warnings

warnings.filterwarnings(action="ignore")
pd.options.display.max_columns = None
# pd.options.display.max_colwidth = None

%matplotlib inline
plt.rcParams['figure.figsize'] = (9, 6)
sns.set(style="darkgrid")

In [2]:
def read_files(dir_path: str, sep: str = ",") -> pd.DataFrame:
    """
    Read the files with timestamps from a given catalog.

    Args:
        dir_path: Path to the catalog with the files
        sep: Separator used in pd.read_csv function. Defaults to ",".

    Returns:
        pd.DataFrame: Data from all the files concatenated into one dataframe.
    """
    files = os.listdir(dir_path)
    data = pd.DataFrame(
        [], columns=["FileId", "timestamp_start", "timestamp_stop", "word"]
    )
    for file in files:
        f = open(os.path.join(dir_path, file), encoding="utf-8", mode="r")
        name = file.split(".")[0]
        df = pd.read_csv(f, header=None, sep=sep, encoding="utf-8")
        df["FileId"] = name
        if sep != ",":
            df["timestamp_start"] = df.iloc[:, 0].str.split(",").str[0].str[1:]
            df["timestamp_stop"] = (
                df.iloc[:, 0].str.split(",").str[1].str.split("\)").str[0]
            )
            df["word"] = df.iloc[:, 0].str.split("\) \s").str[1]
            df.drop([0], axis=1, inplace=True)
            df.drop(df.tail(1).index, inplace=True)
        else:
            df["timestamp_start"] = df.iloc[:, 0].str[1:]
            df["timestamp_stop"] = df.iloc[:, 1].str.split("\) \s").str[0]
            df["word"] = df.iloc[:, 1].str.split("\) \s").str[1]

            df.drop([0, 1], axis=1, inplace=True)
            df.drop(df.tail(1).index, inplace=True)
        data = pd.concat([data, df])
        f.close()
    return data

## <span style='font-family:Georgia'> Data loading & preparation

In [3]:
# reading data
# train
data_train_in = pd.read_csv("../../data/source/train/in.tsv", sep="\t", header=None)
data_train_exp = pd.read_csv(
    "../../data/source/train/expected.tsv", sep="\t", header=None
)
# test
data_test_in = pd.read_csv("../../data/source/test-A/in.tsv", sep="\t", header=None)
data_test_exp = pd.read_csv(
    "../../data/source/test-A/expected.tsv", sep="\t", header=None
)
# train
data_train_clntmstmp = read_files("../../data/source/poleval_fa.train/train", sep="\t")
# test
data_test_clntmstmp = read_files(
    "../../data/source/poleval_fa.validation/validation", sep="\t"
)

In [4]:
# renaming columns
data_train_in.columns = ['FileId', 'ASROutput']
data_train_exp.columns = ['FixedOutput']
data_test_in.columns = ['FileId', 'ASROutput']
data_test_exp.columns = ['FixedOutput']

# adding index to expected output (assumption: the same ordering of the records)
data_train_exp['FileId'] = data_train_in['FileId']
data_train_exp = data_train_exp[['FileId', 'FixedOutput']]

data_test_exp['FileId'] = data_test_in['FileId']
data_test_exp = data_test_exp[['FileId', 'FixedOutput']]

## <span style='font-family:Georgia'> Check for records for which we do not have transcripts with 

In [5]:
# list of file ids
train_exp_file_ids = np.unique(np.array(data_train_exp["FileId"].tolist()))
train_timestamp_file_ids = np.unique(np.array(data_train_clntmstmp["FileId"].tolist()))

test_file_ids = np.unique(np.array(data_test_in["FileId"].tolist()))
test_timestamp_file_ids = np.unique(np.array(data_test_clntmstmp["FileId"].tolist()))

In [6]:
# exclude records for which we do not have transcripts with
train_ids_to_drop = list(
    train_exp_file_ids[~np.isin(train_exp_file_ids, train_timestamp_file_ids)]
)
data_train_in = data_train_in[~data_train_in.FileId.isin(train_ids_to_drop)]
data_train_exp = data_train_exp[~data_train_exp.FileId.isin(train_ids_to_drop)]

In [7]:
data_train_exp.shape

(793, 2)

In [8]:
data_train_exp

,FileId,FixedOutput
0,wikinews228460,"w wywiadzie dla "" polski "" jarosław kaczyński ..."
1,wikinews183471,"dowody prokuratury przedstawiające, że janusz ..."
2,wikinews188040,"w czasie długiego weekendu, 1- 3 maja 2009 w j..."
3,wikinews178804,drugie zwycięstwo w tegorocznym giro d'italia ...
4,wikinews231649,takiego mrozu nie było w polsce od dawna. słup...
...,...,...
795,wikinews188569,piotr małachowski zdobył srebrny medal w rzuci...
796,wikitalks002129,moim zdanie warto jest udoskonalic ten artykol...
797,wikinews183189,we wsi kuleje niedaleko częstochowy piętnastol...
798,wikinews235762,msz wezwało ambasadora rosji siergieja andriej...


## <span style='font-family:Georgia'> Separate special characters from words

In [9]:
data_train_in = separate_special_chars(data_train_in) 
data_train_exp = separate_special_chars(data_train_exp) 
data_test_in = separate_special_chars(data_test_in)
data_test_exp = separate_special_chars(data_test_exp)

In [10]:
data_train_exp["FixedOutput"] = data_train_exp["FixedOutput"].apply(
    rm_consecutive_spaces
)
data_test_exp["FixedOutput"] = data_test_exp["FixedOutput"].apply(
    rm_consecutive_spaces
)
data_train_in["ASROutput"] = data_train_in["ASROutput"].apply(
    rm_consecutive_spaces
)
data_test_in["ASROutput"] = data_test_in["ASROutput"].apply(
    rm_consecutive_spaces
)

## <span style='font-family:Georgia'> Check for records with non-character symbols excluded from target classes set

In [11]:
target_classes=['.', ',', '?', '!', '-', ':', '...']
numbers = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0']

In [12]:
non_characters = np.unique(
    data_train_exp["FixedOutput"].apply(lambda x: find_nonalpha(x))
)
symbols_to_drop = list(
    set(flatten(non_characters)) - set(target_classes) - set(numbers)
)
print(f"List of symbols to be dropped: {str(symbols_to_drop)}")

List of symbols to be dropped: ['β', '(', 'з', 'の', '三', '²', '八', 'ž', 'ã', '·', 'カ', 'э', 'ι', 'ú', 'é', 'л', 'ū', '³', 'け', '尾', '蝓', 'ű', 'ό', '五', 'ю', 'α', 'ê', 'р', 'к', 'ç', "'", 'š', 'í', '%', 'ニ', 'у', 'я', 'г', 'е', 'о', ')', 'й', 'т', ';', 'ν', '€', 'ß', 'и', 'ρ', 'á', 'а', 'в', 'ω', 'φ', 'п', 'ο', 'ä', 'б', 'м', '亀', 'ル', 'х', 'д', 'à', 'ε', 'δ', ' ', 'č', ']', 'ü', 'è', 'ς', 'ő', 'н', '[', 'ř', 'ж', 'ô', 'イ', 'ö', 'ø', '"', 'с', '大', '馬', '蛞', '+', '化', '六']


In [13]:
symbols_to_replace = ["'", '"', ";", "%", "(", ")", "[", "]", "²", "€", "³", "·"]

for symb in symbols_to_replace:
    data_train_exp["FixedOutput"] = data_train_exp["FixedOutput"].apply(
        lambda x: x.replace(symb, "")
    )
    data_train_in["ASROutput"] = data_train_in["ASROutput"].apply(
        lambda x: x.replace(symb, "")
    )

data_train_exp["FixedOutput"] = data_train_exp["FixedOutput"].apply(
    rm_consecutive_spaces
)
data_train_in["ASROutput"] = data_train_in["ASROutput"].apply(rm_consecutive_spaces)

In [14]:
with open(
    "../../data/outputs/eda/train/symbols_to_replace.txt", "w", encoding="utf-8"
) as outfile:
    outfile.write("\n".join(symbols_to_replace))

In [15]:
symbols_to_drop = list(set(symbols_to_drop) - set(symbols_to_replace) - set(" "))
str(symbols_to_drop)  # list of weird letters

"['β', 'の', 'з', '三', '八', 'ž', 'ã', 'カ', 'э', 'ι', 'ú', 'é', 'л', 'ū', 'け', '尾', '蝓', 'ű', 'ό', '五', 'ю', 'α', 'ê', 'р', 'к', 'ç', 'š', 'í', 'ニ', 'у', 'я', 'г', 'е', 'о', 'т', 'й', 'ν', 'ß', 'и', 'ρ', 'á', 'а', 'в', 'ω', 'φ', 'п', 'ο', 'ä', 'б', 'м', '亀', 'ル', 'х', 'д', 'à', 'ε', 'δ', 'č', 'ü', 'è', 'ς', 'ő', 'н', 'ř', 'ж', 'ô', 'イ', 'ö', 'ø', 'с', '大', '馬', '蛞', '+', '化', '六']"

In [16]:
with open(
    "../../data/outputs/eda/train/non_polish_letters.txt", "w", encoding="utf-8"
) as outfile:
    outfile.write("\n".join(symbols_to_drop))

In [17]:
noise = "|".join(r"{}".format(x) for x in symbols_to_drop)  # noise to drop
noise = noise.replace('+', '\+')
# data with noise (train)
data_train_exp_noise = data_train_exp[data_train_exp["FixedOutput"].str.contains(noise)]
data_train_in_noise = data_train_in[data_train_in["ASROutput"].str.contains(noise)]
# data_train_exp_noise.to_csv("../../data/outputs/eda/train/train_noise.csv")

data_train_exp_noise.head()

,FileId,FixedOutput
11,wikinews229879,10 stycznia 2008 r . maria kaczyńska uczestnic...
16,wikinews226894,w dniach 31 maja - 2 czerwca 2013 na terenie k...
19,wikinews227330,adam małysz zajął 54 miejsce podczas niedzieln...
34,wikitalks0013853,co to znaczy promieniowanie zlozone ? o ile wi...
43,wikinews190507,w łodzi może powstać miejski program bezpiecze...


In [18]:
count_punctuation(data_train_exp_noise.set_index('FileId'), 'FixedOutput')

,fullstop,comma,question_mark,exclamation_mark,hyphen,colon,ellipsis
FileId,,,,,,,
wikinews229879,10.0,32.0,0.0,0.0,5.0,3.0,0.0
wikinews226894,11.0,17.0,0.0,0.0,10.0,1.0,0.0
wikinews227330,13.0,10.0,0.0,0.0,2.0,0.0,0.0
wikitalks0013853,7.0,9.0,3.0,0.0,4.0,1.0,0.0
wikinews190507,6.0,10.0,0.0,0.0,10.0,1.0,0.0
...,...,...,...,...,...,...,...
wikinews235478,8.0,19.0,0.0,0.0,2.0,1.0,0.0
wikinews227295,13.0,9.0,0.0,0.0,6.0,5.0,0.0
wikitalks009701,17.0,21.0,5.0,0.0,4.0,1.0,0.0


In [19]:
records_with_noise = np.unique(np.array(data_train_exp_noise['FileId'].tolist()))
records_with_noise

array(['wikinews178430', 'wikinews181681', 'wikinews182385',
       'wikinews182796', 'wikinews182991', 'wikinews183040',
       'wikinews184315', 'wikinews184895', 'wikinews185449',
       'wikinews185969', 'wikinews186002', 'wikinews186130',
       'wikinews186309', 'wikinews186989', 'wikinews187460',
       'wikinews187846', 'wikinews189463', 'wikinews190354',
       'wikinews190502', 'wikinews190507', 'wikinews195039',
       'wikinews195323', 'wikinews195350', 'wikinews195546',
       'wikinews196212', 'wikinews199814', 'wikinews208312',
       'wikinews209076', 'wikinews220334', 'wikinews224132',
       'wikinews226384', 'wikinews226894', 'wikinews227295',
       'wikinews227330', 'wikinews229879', 'wikinews229995',
       'wikinews230757', 'wikinews230782', 'wikinews231078',
       'wikinews231671', 'wikinews234543', 'wikinews235478',
       'wikinews239105', 'wikitalks0010570', 'wikitalks0012072',
       'wikitalks0012266', 'wikitalks0012936', 'wikitalks0013026',
       'wikita

## <span style='font-family:Georgia'> Non-character symbols statistics summary

Count:
* how many symbols_to_drop contain each record
* how long is text in each record
* percent of symbols_to_drop in whole string in each record


In [20]:
# count number of symbols_to_drop in each record
fixed_output= data_train_exp_noise.loc[:,"FixedOutput"]
count=[0 for i in range(len(fixed_output))]
number=0
for string in fixed_output:
    for letter in string:
        if letter in symbols_to_drop:
            count[number] +=1
    number+=1 #iterate for indexes, as from series only strings, without indexes
    
# count length of fixed output
length=[0 for i in range(len(fixed_output))]
number=0
for string in fixed_output:
    length[number]=len(string)
    number+=1
    
df_fixed_output=pd.DataFrame(fixed_output)
df_fixed_output['length']=length
df_fixed_output['count of symbols_to_drop']=count
df_fixed_output['percent']=[i/j*100 for i,j in zip(count, length)]

df_fixed_output.head()

,FixedOutput,length,count of symbols_to_drop,percent
11,10 stycznia 2008 r . maria kaczyńska uczestnic...,2028,2,0.098619
16,w dniach 31 maja - 2 czerwca 2013 na terenie k...,1586,1,0.063052
19,adam małysz zajął 54 miejsce podczas niedzieln...,1195,1,0.083682
34,co to znaczy promieniowanie zlozone ? o ile wi...,1021,2,0.195886
43,w łodzi może powstać miejski program bezpiecze...,1552,3,0.193299


In [21]:
# print whole records, which have at least 1% of characters to drop
to_print=df_fixed_output[df_fixed_output['percent']>1]
for i in range(len(to_print)):
    print(to_print.iloc[i,:].FixedOutput)
    print('\n')

ogoniasty ? czy to jest poprawne ? czy ktoś może wie jak to brzmi w mandze ? konrad mów ! w mandze pisze bijuu , co oznacza demony z ogonami , czyli w sensowniejszym przetłumaczeniu ogoniaste demony . jeżeli potrafi ktoś przetłumaczyć to inaczej to bym poprosił . jest to w pewnym sensie niedokładne przetłumaczenie , ale innego polacy nie znajdą . po pierwsze - znak oznacza bestię , a nie demona . po drugie - słowo ogoniaste przypomina mi słowotwórczość swoistą dla małych dzieci . nie wiem czy taka forma jest poprawna w języku polskim . vigilium hachibi hej , od kiedy to hachibi jest złapany ? zresztą nie tylko hachibi . niedawno pisało tu , że jest prawdopodobnie wolny , a o ile wiem jak na razie nigdzie nie pisało że ktoś go złapał . choć mogę się mylić . nawet nie pisze prawdopodobnie że jest złapany , tylko że na pewno . jego status jest taki sam jak kyuubiego więc nie można napisać , że jest celem sasuke bo przy kyuubim nie ma napisanego , że jest celem paina nazwy bestii pięć z po

In [22]:
# Words containing non-characters symbols preview

In [23]:
count_words=['' for i in range(len(fixed_output))]
number=0
for string in fixed_output:
    words_in_string=string.split(' ')
    for word in words_in_string:
        count_letters=0
        for letter in word:
            if letter in symbols_to_drop:
                count_letters+=1
        if count_letters>0:
            count_words[number]=count_words[number]+' '+word
    number+=1
    
count_words = list(map(str.lstrip, count_words))
print(count_words)

['kadlčika kadlčik', 'motörhead', 'hautamäkim', 'β α', 'fédération française', '+', 'δ', 'bošnjaci', '+ +', 'räikkönena jérôme pérez', 'française rémy grégorio', 'ramírez domínguez', 'congrès', 'güntera', 'όνειρος ω', 'kölner kölner kőln kőln fűr schönberg', 'agustín', 'élysées', 'bermúdez', '+ trentemøller väth', '+ +', '+', 'josé rodríguez', '+ +', 'für', 'ニ尾の化け 三尾の大亀 五尾のイルカ馬 六尾の蛞蝓 八尾の kyogyū', '+ + + +', 'βy', 'φ φ φ φ', 'государственная дума российской империи б ю иванов а а комзолова и с ряховская москва росспэн', '+', 'förstera', 'katajamäki häkkinen räikkönen järvilehto järvilehto räikkönen häkkinen', 'ß ß ß', 'josé durão', '+ +', 'räikkönen räikkönenowi', 'hautamäki ljøkelsøy', 'geração à', 'jános kárászy', 'kovačević kovačević podraščanin miloš saša sánchez díaz tomás', '+', 'изображение', 'püolkim', '+ +', 'maestà maestà maestà maestà maestà', '+', 'székesfehérvár', 'klöden klöden', 'côte saône côte', '+', 'α α', 'újváry', 'irénée', 'angoulême léger angoulême', '+ +', 'görges

In [24]:
with open("../../data/outputs/eda/train/noisy_words.txt", "w", encoding="utf-8") as outfile:
    outfile.write("\n".join(count_words))

In [25]:
pd.options.display.max_colwidth = None

df_fixed_output['words containing']=count_words
df_fixed_output

,FixedOutput,length,count of symbols_to_drop,percent,words containing
11,"10 stycznia 2008 r . maria kaczyńska uczestniczyła w konferencji w belwederze w związku z kampanią medialną jestem przedstawicielem mniejszości narodowej - jestem polakiem , którą objęła honorowym patronatem . kampania została zorganizowana przez związek gmin wyznaniowych żydowskich w rp w ramach europejskiego roku równych szans . podczas konferencji maria kaczyńska zwróciła się do obecnych : witam państwa serdecznie na dzisiejszej konferencji prasowej , a szczególnie serdecznie witam pana piotra kadlčika , przewodniczącego związku gmin wyznaniowych żydowskich w rp - organizatora kampanii - pana macieja bohosiewicza , współprzewodniczącego komisji wspólnej rządu i mniejszości narodowych i etnicznych w polsce , panią mariolę abkowicz , przewodniczącą związku karaimów polskich , a także pomysłodawców kampanii panią malkę kafkę i pana jana geberta i szefa kampanii pana mikea urbaniaka . witam wszystkich przybyłych gości . wierzę , że kampania ta pomoże przełamać stereotyp , jakże niesłuszny , nieżyczliwego postrzegania niektórych mniejszości narodowych i etnicznych , dumnych ze swojej tożsamości , swojego pochodzenia , historii , kultury , religii i tradycji . przecież oni to my , obywatele i mieszkańcy rzeczypospolitej polskiej . w konferencji uczestniczyli m . in : piotr kadlčik , przewodniczący związku gmin wyznaniowych żydowskich w rp , maciej bohosiewicz , współprzewodniczący komisji wspólnej rządu i mniejszości narodowych i etnicznych , mariola abkowicz , przewodnicząca związku karaimów polskich , mike urbaniak , szef kampanii jestem przedstawicielem mniejszości narodowej - jestem polakiem , berenika anders , dyrektor departamentu ds . kobiet , rodziny i przeciwdziałania dyskryminacji w ministerstwie pracy i polityki społecznej , łukasz adam gabler z mpips , osoby , które użyczyły swojego wizerunku w kampanii : elżbieta maria tarkowska chatila , maria stepan , anna ciszewska , magdalena dercz i joanna talewicz - kwiatkowska inni przedstawiciele mniejszości narodowych i etnicznych w polsce .",2028,2,0.098619,kadlčika kadlčik
16,"w dniach 31 maja - 2 czerwca 2013 na terenie kampusu sggw w warszawie odbędzie się 30 edycja festiwalu ursynalia . jest to jeden z największych festiwali studenckich w europie . organizatorem imprezy są samorząd studentów szkoły głównej gospodarstwa wiejskiego oraz od 2009 roku fundacja bonum i arena live production . znane są już główne gwiazdy festiwalu - są to zespoły motörhead oraz zz top , który przyjeżdża do polski po raz pierwszy od ponad 16 lat . na festiwalu wystąpią też him , bullet for my valentine , gentelman , steve aoki , pendulum , parkway drive , soilwork oraz luxtorpeda , hunter , jelonek i mesajah . w ramach przygotowań do festiwalu , w pierwszej połowie kwietnia odbędzie się konkurs ursynalia the tour , przeznaczony dla młodych zespołów rockowych . zgłoszenia można było nadsyłać do 28 marca - komisja konkursowa wybierze z nich 30 zespołów , które zagrają na 6 koncertach eliminacyjnych w różnych miastach polski . każdy z zespołów w czasie koncertu eliminacyjnego może zaprezentować dwa wybrane , autorskie utwory . nagrodą główną , którą otrzymają trzy najlepsze zespoły , jest występ przed headlinerem festiwalu - każdego dnia wystąpi inny zespół , przed inną gwiazdą . konkurs jest prowadzony we współpracy z radiem eska rock . koncerty eliminacyjne odbędą się w 6 miastach : 0804 - warszawa proxima 0904 - gdańsk parlamant 10 4 - łódź wytwórnia 14 4 - katowice megaclub 15 4 - wrocław alibi 16 4 - poznań eskulap na każdy z koncertów wystąpią jako gwiazdy wieczoru zespoły luxtorpeda i jeden z laureatów zeszłorocznej edycji konkursu , zespół chemia .",1586,1,0.063052,motörhead
19,"adam małysz zajął 54 miejsce podczas niedzielnego konkursu pucharu świata na skoczni w oslo - holmenkollen polak oddał skok na odległość 89 m , uzyskując notę 54 , 7 punktu . słaby występ skoczka z wisły spowodowany był podmuchem wiatru , ja